In [1]:
import os
import openai

from decouple import Config, RepositoryEnv
config = Config(RepositoryEnv("/workspaces/codespaces-jupyter/.env"))

openai.api_key = config("OPENAI_API_KEY")

In [2]:
def create_dish_prompt(list_of_ingreds):
    prompt = f"Create a detailed recipe based on only the following ingredients: {', '.join(list_of_ingreds)}.\n" \
        + f"Additionally, assign a title starting with 'Recipe Title: ' to this recipe."

    return prompt

In [5]:
recipe = create_dish_prompt(["ham", "turkey", "eggs", "bread"])
print(recipe)

Create a detailed recipe based on only the following ingredients: ham, turkey, eggs, bread.
Additionally, assign a title starting with 'Recipe Title: ' to this recipe.


In [7]:
response = openai.Completion.create(engine="text-davinci-003",
                                    prompt=recipe,
                                    max_tokens=512,
                                    temperature=0.7)

In [9]:
result_text = response['choices'][0]['text']
print(result_text)



Recipe Title: Ham and Turkey Frittata Sandwich

Ingredients: 
- 4 slices of bread
- 4 eggs
- 4 ounces of deli ham
- 4 ounces of deli turkey

Instructions:
1. Preheat oven to 350 degrees Fahrenheit.
2. Crack the eggs into a large bowl and whisk together until foamy.
3. Add the ham and turkey to the bowl and stir until everything is evenly incorporated.
4. Grease a 9-inch baking dish with butter or cooking spray.
5. Pour the egg mixture into the dish and spread it evenly.
6. Place the dish in the preheated oven and bake for 25 minutes, or until the top is lightly golden.
7. Remove the dish from the oven and let cool for at least 10 minutes.
8. Cut the frittata into 4 equal pieces.
9. Toast the bread in a toaster or in a skillet over medium heat. 
10. Place one piece of the frittata on each slice of toasted bread.
11. Serve and enjoy!


In [10]:
import re

In [11]:
result_text.split('Recipe Title: ')

['\n\n',
 'Ham and Turkey Frittata Sandwich\n\nIngredients: \n- 4 slices of bread\n- 4 eggs\n- 4 ounces of deli ham\n- 4 ounces of deli turkey\n\nInstructions:\n1. Preheat oven to 350 degrees Fahrenheit.\n2. Crack the eggs into a large bowl and whisk together until foamy.\n3. Add the ham and turkey to the bowl and stir until everything is evenly incorporated.\n4. Grease a 9-inch baking dish with butter or cooking spray.\n5. Pour the egg mixture into the dish and spread it evenly.\n6. Place the dish in the preheated oven and bake for 25 minutes, or until the top is lightly golden.\n7. Remove the dish from the oven and let cool for at least 10 minutes.\n8. Cut the frittata into 4 equal pieces.\n9. Toast the bread in a toaster or in a skillet over medium heat. \n10. Place one piece of the frittata on each slice of toasted bread.\n11. Serve and enjoy!']

In [16]:
def extract_title(recipe):
    return re.findall('^.*Recipe Title: .*$', recipe, re.MULTILINE)[0].split('Recipe Title: ')[-1]

In [25]:
recipe_title = extract_title(result_text)
recipe_title

'Ham and Turkey Frittata Sandwich'

In [26]:
def dalle2_prompt(recipe_title):
    prompt = f"{recipe_title}, professional food photography, 15mm, studio lighting"
    return prompt

In [27]:
dalle2_prompt(recipe_title)

'Ham and Turkey Frittata Sandwich, professional food photography, 15mm, studio lighting'

In [29]:
img_response = openai.Image.create(prompt=dalle2_prompt(recipe_title),
                                   n=1,
                                   size='1024x1024')

In [31]:
image_url = img_response['data'][0]['url']

In [23]:
import requests
import shutil

def save_image(image_url, file_name):
    image_res = requests.get(image_url, stream=True)
    if image_res.status_code == 200:
        with open(file_name, 'wb') as f:
            shutil.copyfileobj(image_res.raw, f)
    else:
        print("ERROR LOADING IMAGE!")

    return image_res.status_code



In [32]:
save_image(image_url, 'example_with_more_kw_download.png')

200